# Data Visualization Tutorial: The `folium` Library

## Alex Adams

## November 6, 2021

## PPOL563 Data Visualization for Data Science

There are many different tools for visualizing spatial or geographic data, from the `sf` library in R to the maps in Tableau. However, the `folium` library for Python stands out for being both easy to use and capable of producing high-quality maps. `folium` is a Python interface for the `leaflet` library, a common library in JavaScript for making customizable, interactive maps. `folium` can easily work with many common leaflet plugins and features as well; we'll see some examples of that later as we build our own map.

To start plotting in `folium`, we'll need to install the library with `pip install folium` in the command line. We can then import it in our notebooks and scripts, and get to work. I'm also importing `pandas` (for data manipulation) and `Nominatim` from the `geocoders` module of the `geopy` package, as well as the `features` and `plugins` modules from `folium`. 

In [1]:
import folium
from folium import features
from folium import plugins
from geopy.geocoders import Nominatim
import pandas as pd
app = Nominatim(user_agent="FoliumTutorial")

I've also created an instance of the Nominatim geocoder called `FoliumTutorial` in this setup chunk. The Nominatim geocoder limits the number of queries we can run in a certain length of time, so specifying a user agent name lets the geocoder track our queries.

For this tutorial, we're going to create a map of the metro system in Washington, D.C. using data from the Washington Metropolitan Area Transit Authority Rail Station Information API. I've already extracted the data and converted it from a JSON dictionary to a .csv (along with a few other pre-processing steps to make plotting easier). The D.C. metro consists of 95 stations across six lines, each named for a color (Red, Orange, Yellow, Green, Blue, and Silver).

To create a `folium` map, we only need one piece of information (well, technically two): a set of latitude-longitude coordinates for the approximate center of our map. We can use the geocoder to find the coordinates we need:

In [2]:
location = app.geocode('Washington, D.C.').raw

In [3]:
location

{'place_id': 85088352,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'node',
 'osm_id': 8393724146,
 'boundingbox': ['38.9073822', '38.9074822', '-77.0351422', '-77.0350422'],
 'lat': '38.9074322',
 'lon': '-77.0350922',
 'display_name': 'Embassy of Hungary, 1500, Rhode Island Avenue Northwest, Logan Circle/Shaw, Dupont Circle, Washington, District of Columbia, 2005, United States',
 'class': 'office',
 'type': 'diplomatic',
 'importance': 0.6398478578672753}

The Hungarian Embassy in Washington, D.C. will do just fine. The output of the geocoder query is a dictionary object with location information, including the latitude and longitude of the place we queried.

We can now create a `Map` object using the latitude and longitude from our location dictionary. `folium` maps take in latitude and longitude as a tuple, with latitude first. Let's name our map object `m` and display it.

In [4]:
m = folium.Map(location=[location['lat'], location['lon']])

In [5]:
m

By default, `folium` uses OpenStreetMap for its mapping tiles. It's possible to use other map tiles, but OpenStreetMap works well for the map we're going to make. Right now, our map is centered on D.C., and has a zoom control in the top-left corner, but that's about it. Let's add more detail to this map. First, let's read in a data set of D.C. metro stations and locations:

In [6]:
metro = pd.read_csv('https://raw.githubusercontent.com/aadams149/portfolio/main/Coursework/dc_metro_lines.csv')

Now, let's see what columns we have:

In [7]:
metro.columns

Index(['Code', 'Name', 'StationTogether1', 'StationTogether2', 'LineCode1',
       'LineCode2', 'LineCode3', 'LineCode4', 'LineCode5', 'Lat', 'Lon',
       'Address.Street', 'Address.City', 'Address.State', 'Address.Zip',
       'Line', 'Order'],
      dtype='object')

The most important columns for this map are `Name`, `Lat`, `Lon`, `Line`, `Order`, and the five `LineCode` columns. `Line` and `Order` do not exist in the original data; they've been constructed for this tutorial. Now that we have our data, we can add markers to our map using `folium.Marker`. Similarly to creating the original map object, we need to specify the location as a latitude-longitude tuple. Since `folium` doesn't support vectorized operations (i.e. we can't just write `folium.Marker([metro['lat'],metro['lon']])`, we'll use a `for` loop to add the markers to our map:

In [8]:
#Use a `for` loop and indexing to add each marker to the map individually
for i in range(0, len(metro)):
    folium.Marker([metro['Lat'][i],
                   metro['Lon'][i]]).add_to(m)

What does our map look like now? Let's see.

In [9]:
m

We've added markers, but they don't tell us anything useful. Let's fix that. 

First, we'll redefine our map to the original map of D.C. without markers. Without this step, the new markers we're about to add would be layered on top of the plain blue ones we just added, and that might look strange (not to mention eat up some memory). 

We can use a more complex `for` loop and `if` statements to specify the color of our markers. We can also change the symbol on each icon from a plain dot to something more descriptive. Let's use a train, from the Font Awesome icon library (https://fontawesome.com/v4.7/icons/). The loop below checks the value of the `Line` column for each row of the data frame, and then adds a marker with a specified color and icon at that location.

`folium` only supports certain colors for markers. Red, orange, blue, and green are in that list, so we're good there. Silver isn't supported, but gray is, so we'll use that. The closest option to yellow is beige, which is good enough for our purposes. 

There's one other key detail in this loop: the `popup` argument. For each marker, we specify that `popup` should equal the station name. What does that mean? We'll see after we add these markers.

In [10]:
#Re-instantiate the blank map object
m = folium.Map(location=[location['lat'], location['lon']])

In [11]:
#This `for` loop might look complex, but it's really not!
#Like before, we use indexing to iterate over the data frame.
for i in range(0, len(metro)):
    
    #Check if the value in the 'line' column equals 'Red'
    if metro['Line'][i] == 'Red':
        
        #Plot a marker at the latitude and longitude in that row
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                      
                      #Specify the name of the station as a pop-up
                       popup = metro['Name'][i],
                      
                      #Color the marker red, and change the icon to the Font Awesome Train
                       icon = folium.Icon(color = 'red',
                                          icon = 'train',
                                          prefix = 'fa')).add_to(m) #Then add it to the map
        
    #Next, repeat the process with the other five lines
    elif metro['Line'][i] == 'Blue':
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = metro['Name'][i],
                       icon = folium.Icon(color = 'blue',
                                          icon = 'train',
                                          prefix = 'fa')).add_to(m)
    elif metro['Line'][i] == 'Yellow':
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = metro['Name'][i],
                       icon = folium.Icon(color = 'beige',
                                          icon = 'train',
                                          prefix = 'fa')).add_to(m)
    elif metro['Line'][i] == 'Orange':
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = metro['Name'][i],
                       icon = folium.Icon(color = 'orange',
                                          icon = 'train',
                                          prefix = 'fa')).add_to(m)
    elif metro['Line'][i] == 'Green':
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = metro['Name'][i],
                       icon = folium.Icon(color = 'green',
                                          icon = 'train',
                                          prefix = 'fa')).add_to(m)
    elif metro['Line'][i] == 'Silver':
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = metro['Name'][i],
                       icon = folium.Icon(color = 'gray',
                                          icon = 'train',
                                          prefix = 'fa')).add_to(m)

Now, let's display our map again and see what's different:

In [12]:
m

The markers are all different colors now, and they show a train instead of a dot. But that's not all! Try clicking on a marker. A bubble pops up with the name of the station. This is because of the `popup` argument we used earlier, and it adds extra information to our map. Now, we can see where each station is, its name, and what line it's on.

Well, that last part isn't *quite* true. Many stations in the D.C. metro system serve more than one line; L'Enfant Plaza is on five of them! Right now, however, we only see one marker per station, and we don't have a way to check which other lines run through that station. 

Fortunately, we can fix this! All we need to do are specify some feature groups. Feature groups are more or less what they sound like: groups of features (map elements) we can add to our map. Let's set up seven feature groups, one for each line and one for all the lines togther. We should also re-instantiate our map to remove the markers from before, and add one additional line: `folium.LayerControl().add_to(m)`. Like with the popups, this is important, but we'll see what it does later. We should specify a name for each `FeatureGroup` object to indicate what it is, and then add it to `m`, our map.

In [13]:
#Re-instantiate the blank map object
m = folium.Map(location=[location['lat'], location['lon']])

In [14]:
#Create new feature group objects, name them, and add them to the map
red = folium.FeatureGroup(name = 'Red Line').add_to(m)
orange = folium.FeatureGroup(name = 'Orange Line').add_to(m)
yellow = folium.FeatureGroup(name = 'Yellow Line').add_to(m)
green = folium.FeatureGroup(name = 'Green Line').add_to(m)
blue = folium.FeatureGroup(name = 'Blue Line').add_to(m)
silver = folium.FeatureGroup(name = 'Silver Line').add_to(m)
allstops = folium.FeatureGroup(name = 'All WMATA Stops').add_to(m)

In [15]:
#Add a layer control to the map
folium.LayerControl().add_to(m)

Now that we've created our feature groups, we can add markers to them. The below code might look like a copy of our loop from before, but there are three key differences. One, now each `folium.Marker` statement is wrapped in a `FeatureGroup.add_child()` function. This adds the marker to the specific feature group where we want it, rather than directly adding it to the map. Two, we no longer end our `folium.Marker` calls with `add_to(m)`. This is because we're adding our markers to feature groups, which we've already added to the map in the previous code chunk. Three, our first section of the loop doesn't use an `if` statement. This section adds markers to the "All Stations" feature group. Since we want this group to include all stations on all lines, we don't need to test for any matching values. 


In [16]:
#Similar for-loop as before
for i in range(0, len(metro)):
    #Add each marker to the "All Stations" feature group.
    allstops.add_child(
        folium.Marker([metro['Lat'][i],
                       metro['Lon'][i]],
                     popup = metro['Name'][i],
                     icon = folium.Icon(color = 'purple',
                                       icon = 'train',
                                       prefix = 'fa')))
    if metro['Line'][i] == 'Red':
        #Instead of adding the marker to the map, we add the marker to the feature group.
        #The feature group is already added to the map.
        #This lets us keep each set of stations as a separate layer.
        red.add_child(
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = metro['Name'][i],
                       icon = folium.Icon(color = 'red',
                                          icon = 'train',
                                          prefix = 'fa')))
    #Repeat for the other lines and feature groups
    if metro['Line'][i] == 'Blue':
        blue.add_child(
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = metro['Name'][i],
                       icon = folium.Icon(color = 'blue',
                                          icon = 'train',
                                          prefix = 'fa')))
    if metro['Line'][i] == 'Yellow':
        yellow.add_child(
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = metro['Name'][i],
                       icon = folium.Icon(color = 'beige',
                                          icon = 'train',
                                          prefix = 'fa')))
    if metro['Line'][i] == 'Green':
        green.add_child(
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = metro['Name'][i],
                       icon = folium.Icon(color = 'green',
                                          icon = 'train',
                                          prefix = 'fa')))
    if metro['Line'][i] == 'Orange':
        orange.add_child(
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = metro['Name'][i],
                       icon = folium.Icon(color = 'orange',
                                          icon = 'train',
                                          prefix = 'fa')))
    if metro['Line'][i] == 'Silver':
        silver.add_child(
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = metro['Name'][i],
                       icon = folium.Icon(color = 'gray',
                                          icon = 'train',
                                          prefix = 'fa')))

Let's see what our map looks like now:

In [17]:
m

Whoa, everything's purple! 

This is because we created a feature group of all stations and colored the markers purple. Since that was the last feature group we added to the map, it got layered on top. 

Move your cursor over the icon in the top right which looks like several tiles layered on one another. This is our layer control, and we added it to our map with the `folium.LayerControl()` function call earlier. It shows all the feature groups we've added to our map. Right now, all seven should be checked. Try unchecking the "All WMATA Stops" box. The map should now look like the one we made before.

Try unchecking everything, and then rechecking the Yellow Line. You should see markers running approximately north to south. Now recheck the Green Line. Many of the Yellow Line markers are no longer visible, because they overlap with the Green Line, and the most recently selected feature takes precedence. If you uncheck and recheck the Yellow Line, it should show on top of the Green Line. And of course, if you click on a station, that station name appears in a bubble like before.

This is pretty good! But we can do even better. Right now we can only see the separate stations. What if we want to see how they connect? We can use `folium.PolyLine` to plot our routes. `PolyLine` plots a line from a vector of latitude-longitude tuples. Let's create a new column in our `metro` data frame called `latlong`, consisting of tuples of latitude-longitude coordinates:

In [18]:
metro['latlong'] = list(zip(metro['Lat'],metro['Lon']))

Once we've done that, the rest is fairly simple. We can re-instantiate our map, and then add our stops and lines to it like before. Since we're not going to re-specify our feature groups of stops from before, we don't need to re-run that `for` loop, and we can add the groups to our map with `Map.add_child(featureGroup)`.

In [19]:
#Re-instantiate the blank map object
m = folium.Map(location=[location['lat'], location['lon']])

In [20]:
#Add the feature groups using add_child
m.add_child(red)
m.add_child(orange)
m.add_child(yellow)
m.add_child(green)
m.add_child(blue)
m.add_child(silver)
m.add_child(allstops)

#Add the layer control object
folium.LayerControl().add_to(m)

Now let's add our `PolyLine` layers. For each line, we want to select the `latlong` column from `metro`, but only where the `Line` equals a specific value. This is why I added the `Order` column to this dataset. `PolyLine` plots the line in the order it receives coordinates, so if the coordinates aren't in the right order, the line might end up doubling back on itself or having weird angles. If you've rearranged the data set at all, you can sort it by `Line` and `Order` to get the stations back in the correct order. Like with the station markers, we're specifying a pop-up with the line name, but we're also adding a tool tip. After we create the map, move your cursor over a line and see what appears! 

In [21]:
#Create a line using the latlong tuples and dataframe filtering methods
folium.PolyLine(
    metro[metro['Line'] == 'Red']['latlong'],
    #Specify a color, thickness, opacity, text to appear on clicking, and text for mouse-over
    color="red", weight=4, opacity=1, popup = 'Red Line', tooltip = 'Red Line').add_to(m) #Then add the line to the map

#Repeat for the other five lines
folium.PolyLine(
    metro[metro['Line'] == 'Orange']['latlong'],
    color="orange", weight=4, opacity=1, popup = 'Orange Line', tooltip = 'Orange Line').add_to(m)

folium.PolyLine(
    metro[metro['Line'] == 'Yellow']['latlong'],
    color="beige", weight=4, opacity=1, popup = 'Yellow Line', tooltip = 'Yellow Line').add_to(m)

folium.PolyLine(
    metro[metro['Line'] == 'Green']['latlong'],
    color="green", weight=4, opacity=1, popup = 'Green Line', tooltip = 'Green Line').add_to(m)

folium.PolyLine(
    metro[metro['Line'] == 'Blue']['latlong'],
    color="blue", weight=4, opacity=1, popup = 'Blue Line', tooltip = 'Blue Line').add_to(m)

folium.PolyLine(
    metro[metro['Line'] == 'Silver']['latlong'],
    color="Silver", weight=4, opacity=1, popup = 'Silver Line', tooltip = 'Silver Line').add_to(m)

In [22]:
m

We're almost done! This map communicates much more information than those blue markers with the dots, but there's still a little more we can do to make this even more effective. What if we want to visualize the stops and routes separately? Simple! We just create additional feature groups. Let's create a group for each set of stops and for each route (after re-instantiating our map, of course):

In [23]:
#Re-instantiate a blank map object
m = folium.Map(location=[location['lat'], location['lon']])

In [24]:
#Create two feature groups for each line
#[color]_stops, for the station locations
#[color]_line, for the connecting line
red_stops = folium.FeatureGroup('RD Line Stops').add_to(m)
red_line = folium.FeatureGroup('RD Line Route').add_to(m)

orange_stops = folium.FeatureGroup('OR Line Stops').add_to(m)
orange_line = folium.FeatureGroup('OR Line Route').add_to(m)

yellow_stops = folium.FeatureGroup('YL Line Stops').add_to(m)
yellow_line = folium.FeatureGroup('YL Line Route').add_to(m)

green_stops = folium.FeatureGroup('GR Line Stops').add_to(m)
green_line = folium.FeatureGroup('GR Line Route').add_to(m)

blue_stops = folium.FeatureGroup('BL Line Stops').add_to(m)
blue_line = folium.FeatureGroup('BL Line Route').add_to(m)

silver_stops = folium.FeatureGroup('SV Line Stops').add_to(m)
silver_line = folium.FeatureGroup('SV Line Route').add_to(m)

#Create an additional feature group for all stops in the WMATA system
allstops = folium.FeatureGroup(name = 'All WMATA Stops').add_to(m)

In [25]:
#Add a layer control object
folium.LayerControl().add_to(m)

Before we do that, let's add two more widgets to our map from the `folium.plugins` module we imported earlier. `plugins.Geocoder` will add a search bar, so anyone using our map can search for a specific location. We'll place it in the bottom left of the map, since the zoom controls are in the top left and the layer control is in the top right. We're also going to use `plugins.Fullscreen` to add a full-screen button. This will be attached to the zoom controls, and it lets us expand our map to a full-screen view.

In [26]:
#Add a location search bar
plugins.Geocoder(position = 'bottomleft').add_to(m)

#Add a full screen button
plugins.Fullscreen().add_to(m)

We're also going to modify our `for` loop, but very slightly. Let's add a bit more information to the pop-up for each station. The five `LineCode` columns contain the codes (RD, OR, YL, GR, BL, and SV) for the lines which run through each station. We can use the plus sign `+` to concatenate the station name and line codes together, so that each pop-up will show the station name and the lines it services. Empty line codes are recorded as '---' in the data set, so that's how they'll appear on the map.

In [27]:
#This is largely the same loop as before
for i in range(0, len(metro)):
    allstops.add_child(
        folium.Marker([metro['Lat'][i],
                       metro['Lon'][i]],
                     #The text for the popup is now 'Station Name (Line1, Line2, Line3, Line4, Line5)'
                     popup = (metro['Name'][i]+" ("+metro['LineCode1'][i]+","
                              +metro['LineCode2'][i]+","+metro['LineCode3'][i]
                              +","+metro['LineCode4'][i]+","+metro['LineCode5'][i]+")"),
                     icon = folium.Icon(color = 'purple',
                                       icon = 'train',
                                       prefix = 'fa')))
    #Repeat across all lines
    if metro['Line'][i] == 'Red':
        red_stops.add_child(
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = (metro['Name'][i]+" ("+metro['LineCode1'][i]+","
                              +metro['LineCode2'][i]+","+metro['LineCode3'][i]
                              +","+metro['LineCode4'][i]+","+metro['LineCode5'][i]+")"),
                       icon = folium.Icon(color = 'red',
                                          icon = 'train',
                                          prefix = 'fa')))
    if metro['Line'][i] == 'Blue':
        blue_stops.add_child(
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = (metro['Name'][i]+" ("+metro['LineCode1'][i]+","
                              +metro['LineCode2'][i]+","+metro['LineCode3'][i]
                              +","+metro['LineCode4'][i]+","+metro['LineCode5'][i]+")"),
                       icon = folium.Icon(color = 'blue',
                                          icon = 'train',
                                          prefix = 'fa')))
    if metro['Line'][i] == 'Yellow':
        yellow_stops.add_child(
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = (metro['Name'][i]+" ("+metro['LineCode1'][i]+","
                              +metro['LineCode2'][i]+","+metro['LineCode3'][i]
                              +","+metro['LineCode4'][i]+","+metro['LineCode5'][i]+")"),
                       icon = folium.Icon(color = 'beige',
                                          icon = 'train',
                                          prefix = 'fa')))
    if metro['Line'][i] == 'Green':
        green_stops.add_child(
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = (metro['Name'][i]+" ("+metro['LineCode1'][i]+","
                              +metro['LineCode2'][i]+","+metro['LineCode3'][i]
                              +","+metro['LineCode4'][i]+","+metro['LineCode5'][i]+")"),
                       icon = folium.Icon(color = 'green',
                                          icon = 'train',
                                          prefix = 'fa')))
    if metro['Line'][i] == 'Orange':
        orange_stops.add_child(
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = (metro['Name'][i]+" ("+metro['LineCode1'][i]+","
                              +metro['LineCode2'][i]+","+metro['LineCode3'][i]
                              +","+metro['LineCode4'][i]+","+metro['LineCode5'][i]+")"),
                       icon = folium.Icon(color = 'orange',
                                          icon = 'train',
                                          prefix = 'fa')))
    if metro['Line'][i] == 'Silver':
        silver_stops.add_child(
        folium.Marker([metro['Lat'][i], 
                       metro['Lon'][i]], 
                       popup = (metro['Name'][i]+" ("+metro['LineCode1'][i]+","
                              +metro['LineCode2'][i]+","+metro['LineCode3'][i]
                              +","+metro['LineCode4'][i]+","+metro['LineCode5'][i]+")"),
                       icon = folium.Icon(color = 'gray',
                                          icon = 'train',
                                          prefix = 'fa')))

Finally, we add our lines to their own feature groups.

In [28]:
#Create lines using the same filtering and tuples as before, but add them to feature groups this time
red_line.add_child(folium.PolyLine(
    metro[metro['Line'] == 'Red']['latlong'],
    color="red", weight=4, opacity=1, popup = 'Red Line', tooltip = 'Red Line'))

#Repeat for each [color]_line feature group
orange_line.add_child(folium.PolyLine(
    metro[metro['Line'] == 'Orange']['latlong'],
    color="orange", weight=4, opacity=1, popup = 'Orange Line', tooltip = 'Orange Line'))

yellow_line.add_child(folium.PolyLine(
    metro[metro['Line'] == 'Yellow']['latlong'],
    color="beige", weight=4, opacity=1, popup = 'Yellow Line', tooltip = 'Yellow Line'))

green_line.add_child(folium.PolyLine(
    metro[metro['Line'] == 'Green']['latlong'],
    color="green", weight=4, opacity=1, popup = 'Green Line', tooltip = 'Green Line'))

blue_line.add_child(folium.PolyLine(
    metro[metro['Line'] == 'Blue']['latlong'],
    color="blue", weight=4, opacity=1, popup = 'Blue Line', tooltip = 'Blue Line'))

silver_line.add_child(folium.PolyLine(
    metro[metro['Line'] == 'Silver']['latlong'],
    color="gray", weight=4, opacity=1, popup = 'Silver Line', tooltip = 'Silver Line'))

Let's output our map for the last time, and take in our handiwork:

In [29]:
m

This is pretty cool! We've:
- made a map of the Washington, D.C. metro system,
- placed markers at each station, 
- color-coded those markers, 
- changed the symbol on the marker, 
- added the station name and the lines it services to each marker,
- plotted the routes for each metro line,
- color-coded those routes,
- enabled feature layers so we can see different subsets of stations and routes,
- added a search bar,
- and added a full-screen button.

While this map is a fun side project, it's also legitimately useful (at least if you live in the D.C. area), and we made it relatively quickly using the `folium` package. It's solid as it is, but for an extra bit of pizzazz, swap out `plugins.AntPath` for `folium.PolyLine` when defining the lines above. This accesses the AntPath plugin for `leaflet`, and produces paths which show movement along the lines. This is cool, but can be distracting, so PolyLine was more effective for this particular task. 

Finally, we can export our map to an HTML object (keeping its interactivity) with `Map.save('filename.html')`.

In [30]:
m.save('dc_metro_map.html')